In [3]:
import pandas as pd

In [4]:
data_offline= pd.read_csv('ccf_offline.csv')
data_online = pd.read_csv('ccf_online.csv')

In [20]:
data_online

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513,null
1,13740231,34805,1,null,null,null,20160321
2,14336199,18907,0,null,null,null,20160618
3,14336199,18907,0,null,null,null,20160618
4,14336199,18907,0,null,null,null,20160618
5,14336199,18907,0,null,null,null,20160618
6,14336199,18907,0,null,null,null,20160618
7,14336199,18907,0,null,null,null,20160618
8,14336199,18907,0,null,null,null,20160618
9,14336199,18907,0,null,null,null,20160618


# (1) Online

In [66]:
def get_online_user_table(data_online):
    
    #Number of time this customer gets a coupon online
    a=data_online[['User_id','Coupon_id']]
    mask = data_online['Coupon_id']!='null'
    num_coupon = a[mask].groupby(['User_id']).count().reset_index()
    num_coupon.columns = ['User_id','Tot_coupon_got']
    data_online1 = pd.merge(data_online, num_coupon, on = 'User_id', how='left')

    #Number of time this customer uses a coupon
    mask2 = (data_online1['Date']!= 'null') & (data_online1['Coupon_id']!= 'null')
    b=data_online1[mask2].groupby(['User_id'])['Action'].count().reset_index()
    b.columns = ['User_id','Num_coupon_used']
    data_online2 = pd.merge(data_online1, b, on='User_id', how='left')

    #The average amount of coupon the customer got a week.
    #The training set is collected from Jan. 1st 2016 to June 30th 2016. 26 weeks in total
    def to_float (s):
        return float (s)
    data_online3 = data_online2
    data_online3['Tot_coupon_got']=data_online3['Tot_coupon_got'].map(to_float)
    data_online3['Avg_coupon_got']=data_online3['Tot_coupon_got']*1/26

    #generate customer table
    customer_columns= data_online3[['User_id','Tot_coupon_got','Num_coupon_used','Avg_coupon_got']]
    customer_online_table = customer_columns.drop_duplicates()
    return customer_online_table

In [68]:
get_online_user_table(data_online) [:5]

,User_id,Tot_coupon_got,Num_coupon_used,Avg_coupon_got
0,13740231,6.0,NaN,0.230769
2,14336199,NaN,NaN,NaN
19,10539231,NaN,NaN,NaN
22,14438631,NaN,NaN,NaN
29,15034599,NaN,NaN,NaN


# (2) Offline

In [69]:
def get_offline_customer_table(data_offline):
    #Number of time this customer gets a coupon
    a=data_offline[['User_id','Coupon_id']]
    mask = data_offline['Coupon_id']!='null'
    num_coupon = a[mask].groupby(['User_id']).count().reset_index()
    num_coupon.columns = ['User_id','Tot_coupon_got']
    data_offline1 = pd.merge(data_offline, num_coupon, on = 'User_id',how='left')
    
    #Number of time this customer uses a coupon
    mask2 = (data_offline1['Date']!= 'null') & (data_offline1['Coupon_id']!= 'null')
    b=data_offline1[mask2].groupby(['User_id'])['Coupon_id'].count().reset_index()
    b.columns = ['User_id','Num_coupon_used']
    data_offline2 = pd.merge(data_offline1, b, on='User_id',how='left')
    
    #The average distance of the shops this customer uses a coupon.
    null_mask = data_offline2['Distance']!='null'

    def to_float(s):
        return float(s)

    data_offline3 =data_offline2[null_mask]
    data_offline3['Distance']=data_offline3['Distance'].map(to_float)
    data_offline4 = pd.DataFrame(data_offline3.groupby('User_id')['Distance'].mean()).reset_index()
    data_offline5 = pd.merge(data_offline2, data_offline4,on='User_id',how='left')
    data_offline5=data_offline5.rename(columns = {'Distance_y':'Avg_Distance'})
    
    #The average amount of coupon the customer got a week.
    #The training set is collected from Jan. 1st 2016 to June 30th 2016. 26 weeks in total
    def to_float (s):
        return float (s)
    data_offline6 = data_offline5
    data_offline6['Tot_coupon_got']=data_offline6['Tot_coupon_got'].map(to_float)
    data_offline6['Avg_coupon_got']=data_offline6['Tot_coupon_got']*1/26

    customer_columns2= data_offline6[['User_id','Tot_coupon_got','Num_coupon_used','Avg_Distance','Avg_coupon_got']]
    customer_offline_table = customer_columns2.drop_duplicates()
    return customer_offline_table

In [70]:
get_offline_customer_table(data_offline)[:5]

C:\Users\Jianing Cui\Anaconda2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,User_id,Tot_coupon_got,Num_coupon_used,Avg_Distance,Avg_coupon_got
0,1439408,5.0,1.0,0.142857,0.192308
7,1832624,1.0,NaN,0.000000,0.038462
8,2029232,3.0,NaN,0.200000,0.115385
13,2747744,1.0,NaN,NaN,0.038462
14,196342,1.0,NaN,1.000000,0.038462


# (3) Online Vs Offline

In [73]:
def get_bothline_customer_table(data_online, data_offline):
    #The relative ratio of a customer’s online and offline purchasing within those months
    data_bothline = pd.merge(data_online, data_offline, on='User_id')
    mask1 = data_bothline['Date_x']!='null'
    c=data_bothline[['User_id','Date_x']]
    mask2 = data_bothline['Date_y']!='null'
    d=data_bothline[['User_id','Date_y']]
    
    num_online_purchase = c[mask1].groupby(['User_id']).count().reset_index()
    num_online_purchase.columns = ['User_id','num_online_purchase']
    num_online_purchase['num_online_purchase'].fillna(0,inplace=True)
        
    num_offline_purchase = c[mask2].groupby(['User_id']).count().reset_index()
    num_offline_purchase.columns = ['User_id','num_offline_purchase']
    num_offline_purchase['num_offline_purchase'].fillna(0,inplace=True)
    
    num_bothline = pd.merge(num_online_purchase, num_offline_purchase, on='User_id',how='outer')
    num_bothline['offline/online_ratio'] = num_bothline['num_offline_purchase']/num_bothline['num_online_purchase']
    data_bothline2 = pd.merge(data_bothline,num_bothline,on='User_id',how='left')

    customer_columns3= data_bothline2[['User_id','num_online_purchase','num_offline_purchase','offline/online_ratio']]
    customer_bothline_table = customer_columns3.drop_duplicates()
    return customer_bothline_table

In [74]:
get_bothline_customer_table(data_online, data_offline)[:5]

,User_id,num_online_purchase,num_offline_purchase,offline/online_ratio
0,556864,2.0,NaN,NaN
3,15949,2.0,NaN,NaN
5,5145092,2.0,NaN,NaN
7,4549124,1.0,NaN,NaN
14,1670017,64.0,54.0,0.84375
